![image](https://www.ewatercycle.org/assets/logo.png)

In [14]:
from pathlib import Path
import time
from esmvalcore.experimental import CFG
from ewatercycle import forcing
from ewatercycle.models import Lisflood
from ewatercycle.models.lisflood import reindex_forcings
from ewatercycle.models.lisflood import LisfloodParameterSet

In [15]:
# load config file
CFG.load_from_file('~/.esmvaltool/config-user.yml')

In [16]:
# forcing.generate takes a single forcing dataset
forcing_output = forcing.generate(
    model='lisflood', 
    dataset='ERA-Interim',
    startyear=1990, 
    endyear=1990,
    shapefile='Meuse/Meuse.shp',
)

{'auxiliary_data_dir': PosixPath('/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/Lorentz_Basin_Shapefiles'),
 'compress_netcdf': False,
 'config_developer_file': None,
 'config_file': PosixPath('/home/fakhereh/.esmvaltool/config-user.yml'),
 'drs': {'CMIP5': 'default', 'CMIP6': 'default'},
 'exit_on_warning': False,
 'log_level': 'info',
 'max_parallel_tasks': 1,
 'output_dir': PosixPath('/scratch-shared/ewatercycle'),
 'output_file_type': 'png',
 'plot_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_113450/plots'),
 'preproc_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_113450/preproc'),
 'profile_diagnostic': False,
 'remove_preproc_dir': True,
 'rootpath': {'OBS6': [PosixPath('/lustre1/0/wtrcycle/comparison/obs6')]},
 'run_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_113450/run'),
 'save_intermediary_cubes': False,
 'work_dir': PosixPath('/scratch-shared/ewatercycle/recipe_lisflood_20210413_113450/wor

In [17]:
list(forcing_output.recipe_output.items())

[('diagnostic_daily/script',
  diagnostic_daily/script:
    DataFile('lisflood_ERA-Interim_Meuse_e_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_pr_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tasmin_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_sfcWind_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tasmax_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_rsds_1990_1990.nc')
    DataFile('lisflood_ERA-Interim_Meuse_tas_1990_1990.nc'))]

In [1]:
mask = Path("/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD/catchment_masks.nc")
TEMP_DIR = Path(f"/scratch/shared/ewatercycle/lisflood_{time.strftime('%Y%m%d_%H%M%S')}")
TEMP_DIR.mkdir(parents=True, exist_ok=True)
print(TEMP_DIR)

In [3]:
# reindex forcing to global extent
reindex_forcings(mask, forcing_output, TEMP_DIR)

In [ ]:
parameterset = LisfloodParameterSet(
    # TODO one level down (+ /Lisflood01degree_masked)?
    root=Path('/projects/0/wtrcycle/comparison/lisflood_input'),
    # TODO dir or file (+ /model_mask.nc)?
    mask=mask.parent,
    config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisflood.xml'),
    lisvap_config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisvap.xml'),
)

In [5]:
# create lisflood instance
model = Lisflood()

In [ ]:
# setup model
config_file, config_dir = model.setup(TEMP_DIR, parameterset)

In [6]:
# run lisvap
model.run_lisvap(TEMP_DIR)

In [ ]:
# get metadata
print(model.get_output_var_names())

In [ ]:
# initialize
model.initialize(config_file)

In [ ]:
# run model with bmi
model.update()

In [ ]:
"""singularity exec -B /projects/0/wtrcycle/comparison/lisflood_input,/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD,/scratch/shared/ewatercycle/lisflood_20210413_134355,/scratch/shared/ewatercycle/lisflood_20210413_144250 ewatercycle-lisflood-grpc4bmi.sif python3
Python 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import sys
>>> sys.path.append('/opt/Lisflood')
>>> from bmi_lisflood import LisfloodBmi
>>> model = LisfloodBmi()
>>> model.initialize('/scratch/shared/ewatercycle/lisflood_20210413_144250/lisflood_setting_1990_1990.xml')
>>> model.update()
.Simulation started on 2021-04-13 16:05
1    

 ========================== LISFLOOD Warning =============================
Warning: NaN values on simulated domain in /scratch/shared/ewatercycle/lisflood_20210413_144250/lisflood_et0_1990_1990.nc


 ========================== LISFLOOD Warning =============================
Warning: NaN values on simulated domain in /scratch/shared/ewatercycle/lisflood_20210413_144250/lisflood_e0_1990_1990.nc
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/opt/Lisflood/bmi_lisflood.py", line 72, in update
    self.framework._userModel().dynamic()
  File "/opt/Lisflood/Lisflood_dynamic.py", line 82, in dynamic
    self.snow_module.dynamic()
  File "/opt/Lisflood/hydrological_modules/snow.py", line 112, in dynamic
    TavgS = self.var.Tavg + self.var.DeltaTSnow * (i - 1)
ValueError: operands could not be broadcast together with shapes (256,) (3060,)"""

In [ ]:
# stop the model
model.finalize()
del model.bmi